In [1]:
import sys
import os
from os import path
sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))

In [2]:
from risk_models import *
from risk_models import _name_FT_I_DTL_SEA_PRE_RECORDED, \
    _name_FT_I_DTL_SEA_CONTAINER, _name_FT_I_DTL_COARRI_CTNR, _name_DIM_OPERATOR, _name_DIM_FTZ_CORP, \
    _name_FT_E_DTL_SEA_PRE_RECORDED, \
    _name_FT_E_DTL_SEA_CONTAINER, _name_FT_E_DTL_COARRI_CTNR, _name_MX_BVD, _name_FT_I_DTL_SEA_LIST, _name_FT_E_DTL_SEA_LIST

# clean

In [3]:
child_task_id = 'childtaskidsc1001'
org_code, param_json, base_time = read_log_table(child_task_id)

2021-07-05 10:57:06.869 | INFO     | risk_models.config.read_config.read_func:read_oracle:83 - Read Table successfully! , Total read time spent 0.666s


In [4]:
model_code = 'SUPPLYCHAIN'
child_model_code = 'SC1'
child_task_id = child_task_id
Risk_logger(child_task_id=child_task_id, exec_status=None)

# 参数读取
org_code = org_code
params = json.loads(param_json)
startdt = (dt_method.strptime(base_time, "%Y-%m-%d %H:%M:%S") - datetime.timedelta(days=182)) \
    .strftime('%Y-%m-%d')
enddt = (dt_method.strptime(base_time, "%Y-%m-%d %H:%M:%S") - datetime.timedelta(days=31)) \
    .strftime('%Y-%m-%d')

In [8]:
sql1 = f"""
        SELECT 
        TRADE_NAME,
        E.UNI_SC_ID as ORG_CODE,
        GOODS_OWNER,
        CUS_DECL_AGENT,
        OVERSEAS_CONSIGNOR_NAME_EN,
        D.OPR_NAME_CN,
        SUM(GOODS_GROSS_VALUE_RMB) AS SUM_RMB
        FROM {_name_FT_I_DTL_SEA_PRE_RECORDED} A
        JOIN {_name_FT_I_DTL_SEA_CONTAINER} B  ON A.ID=B.HEAD_id
        JOIN  {_name_FT_I_DTL_SEA_LIST} F ON F.HEAD_ID = A.ID
        LEFT JOIN {_name_FT_I_DTL_COARRI_CTNR} C ON B.VSL_NAME=C.VSL_NAME
        AND B.VOYAGE=C.VOYAGE AND B.CTNR_NO=C.CTNR_NO
        LEFT JOIN {_name_DIM_OPERATOR} D ON C.CTNR_OPR_CODE=D.OPR_CODE
        inner join {_name_DIM_FTZ_CORP} E on a.trade_name = e.corp_name
        where a.decl_date>=to_date('{startdt}','yyyy-mm-dd')
        AND a.decl_date<to_date('{enddt}','yyyy-mm-dd')
        GROUP BY TRADE_NAME,UNI_SC_ID,GOODS_OWNER,CUS_DECL_AGENT,OVERSEAS_CONSIGNOR_NAME_EN,OPR_NAME_CN
        """
sql2 = f"""
        SELECT 
        TRADE_NAME,
        E1.UNI_SC_ID as ORG_CODE,
        GOODS_OWNER,
        CUS_DECL_AGENT,
        OVERSEAS_CONSIGNOR_NAME_EN,
        D1.OPR_NAME_CN,
        SUM(GOODS_GROSS_VALUE_RMB) AS SUM_RMB
        FROM {_name_FT_E_DTL_SEA_PRE_RECORDED} A1
        JOIN {_name_FT_E_DTL_SEA_CONTAINER} B1 ON A1.ID=B1.HEAD_id
        JOIN  {_name_FT_E_DTL_SEA_LIST} F1 ON F1.HEAD_ID = A1.ID
        LEFT JOIN {_name_FT_E_DTL_COARRI_CTNR} C1 ON B1.VSL_NAME=C1.VSL_NAME
        AND B1.VOYAGE=C1.VOYAGE AND B1.CTNR_NO=C1.CTNR_NO
        LEFT JOIN {_name_DIM_OPERATOR} D1 ON C1.CTNR_OPR_CODE=D1.OPR_CODE
        inner join {_name_DIM_FTZ_CORP} E1 on a1.trade_name = e1.corp_name
        where a1.decl_date>=to_date('{startdt}','yyyy-mm-dd')
        AND a1.decl_date<to_date('{enddt}','yyyy-mm-dd')
        GROUP BY TRADE_NAME,UNI_SC_ID,GOODS_OWNER,CUS_DECL_AGENT,OVERSEAS_CONSIGNOR_NAME_EN,OPR_NAME_CN
        """
df_sc1 = Read_Oracle().read_oracle(sql=sql1, database='dbdw')
df_sc2 = Read_Oracle().read_oracle(sql=sql2, database='dbdw')

2021-07-05 10:59:16.216 | INFO     | risk_models.config.read_config.read_func:read_oracle:83 - Read Table successfully! , Total read time spent 15.48s
2021-07-05 10:59:43.869 | INFO     | risk_models.config.read_config.read_func:read_oracle:83 - Read Table successfully! , Total read time spent 27.64s


In [11]:
df_sc = pd.concat([df_sc1,df_sc2],axis=0)

In [12]:
df_sc.head(1)

,TRADE_NAME,ORG_CODE,GOODS_OWNER,CUS_DECL_AGENT,OVERSEAS_CONSIGNOR_NAME_EN,OPR_NAME_CN,SUM_RMB
0,威腾供应链管理（上海）有限公司,913101155820927689,威腾供应链管理（上海）有限公司,上海欣林供应链管理有限公司,HAY ApS,利胜地中海航运（上海）有限公司,1496783.26


In [76]:
df_sc.rename(columns={'SUM_RMB': 'TRADE_VOLUMN'}, inplace=True)
df_sc['TRADE_TIMES'] = 1
df_goods_owner = df_sc.groupby(['TRADE_NAME','ORG_CODE', 'GOODS_OWNER'])['TRADE_TIMES'].count().reset_index()
df_goods_owner_2 = df_sc.groupby(['TRADE_NAME', 'GOODS_OWNER','ORG_CODE'])['TRADE_VOLUMN'].sum().reset_index()
df_goods_owner = df_goods_owner.merge(df_goods_owner_2, how='left', on=['TRADE_NAME','ORG_CODE', 'GOODS_OWNER'])
df_goods_owner['TYPE'] = '货主'
df_goods_owner.rename(columns={'GOODS_OWNER': 'SC_COMPANY'}, inplace=True)
df_agent = df_sc.groupby(['TRADE_NAME','ORG_CODE', 'CUS_DECL_AGENT'])['TRADE_TIMES'].count().reset_index()
df_agent_2 = df_sc.groupby(['TRADE_NAME','ORG_CODE', 'CUS_DECL_AGENT'])['TRADE_VOLUMN'].sum().reset_index()
df_agent = df_agent.merge(df_agent_2, how='left', on=['TRADE_NAME','ORG_CODE', 'CUS_DECL_AGENT'])
df_agent['TYPE'] = '报关行'
df_agent.rename(columns={'CUS_DECL_AGENT': 'SC_COMPANY'}, inplace=True)
df_oversea = df_sc.groupby(['TRADE_NAME', 'ORG_CODE','OVERSEAS_CONSIGNOR_NAME_EN'])['TRADE_TIMES'].count().reset_index()
df_oversea_2 = df_sc.groupby(['TRADE_NAME','ORG_CODE', 'OVERSEAS_CONSIGNOR_NAME_EN'])['TRADE_VOLUMN'].sum().reset_index()
df_oversea = df_oversea.merge(df_oversea_2, how='left', on=['TRADE_NAME', 'ORG_CODE','OVERSEAS_CONSIGNOR_NAME_EN'])
df_oversea['TYPE'] = '境外收发货人'
df_oversea.rename(columns={'OVERSEAS_CONSIGNOR_NAME_EN': 'SC_COMPANY'}, inplace=True)
df_ship_opr = df_sc.groupby(['TRADE_NAME', 'ORG_CODE','OPR_NAME_CN'])['TRADE_TIMES'].count().reset_index()
df_ship_opr_2 = df_sc.groupby(['TRADE_NAME','ORG_CODE', 'OPR_NAME_CN'])['TRADE_VOLUMN'].sum().reset_index()
df_ship_opr = df_ship_opr.merge(df_ship_opr_2, how='left', on=['TRADE_NAME','ORG_CODE', 'OPR_NAME_CN'])
df_ship_opr['TYPE'] = '船公司'
df_ship_opr.rename(columns={'OPR_NAME_CN': 'SC_COMPANY'}, inplace=True)
# 链接所有表CONCAT

frames = [df_goods_owner, df_agent, df_oversea, df_ship_opr]
df_supply_chain = pd.concat(frames, axis=0)

In [77]:
df_supply_chain.head(3)

,TRADE_NAME,ORG_CODE,SC_COMPANY,TRADE_TIMES,TRADE_VOLUMN,TYPE
0,上海上港保税仓储管理有限公司,91310115MA1K4AN53M,上海上港保税仓储管理有限公司,15,5.396253e+09,货主
1,上海东方嘉盛供应链管理有限公司,91310000677818840H,上海东方嘉盛供应链管理有限公司,110,2.889655e+08,货主
2,上海东泽供应链管理有限公司,91310000301610611H,上海东泽供应链管理有限公司,8,7.140588e+06,货主


In [78]:
sql = f"""select bvd.corp_name,rating,bvd_rate,E1.UNI_SC_ID as ORG_CODE from {_name_MX_BVD} bvd left join {_name_DIM_FTZ_CORP} E1 on bvd.corp_name = E1.corp_name"""
df_bvd = Read_Oracle().read_oracle(sql=sql, database='dbods')
df_bvd

2021-06-08 11:28:19.449 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.140s


,CORP_NAME,RATING,BVD_RATE,ORG_CODE
0,ERVIN AMASTEEL淄博安茂福国际贸易有限公司销售,None,NaN,None
1,ESKULANAK GROUPE LAUAK,None,NaN,None
2,EUROCAE,None,NaN,None
3,EVAC AG,None,NaN,None
4,EXPLISEAT SAS,None,NaN,None
...,...,...,...,...
657,"Toray Composite Materials America, Inc.",None,NaN,None
658,Transtar Metals,None,NaN,None
659,Triumph Thermal Systems,None,NaN,None
660,Tronair.Inc,None,NaN,None


In [79]:
sql = f"""select corp_name,rating,bvd_rate from {_name_MX_BVD}"""
df_bvd = Read_Oracle().read_oracle(sql=sql, database='dbods')
df_bvd['TRADE_NAME'] = '上海飞机制造有限公司'
df_bvd['ORG_CODE'] = '91310000132612172J'
df = pd.merge(df_supply_chain, df_bvd, left_on=['TRADE_NAME', 'SC_COMPANY','ORG_CODE'],
              right_on=['TRADE_NAME', 'CORP_NAME','ORG_CODE'], how='outer')
df['SC_COMPANY'] = df['SC_COMPANY'].fillna('')
df.loc[df['SC_COMPANY'] == '', 'SC_COMPANY'] = df.loc[df['SC_COMPANY'] == '', 'CORP_NAME']
df.loc[df['TYPE'].isna(), 'TRADE_TIMES'] = 0
df.loc[df['TYPE'].isna(), 'TYPE'] = '境外收发货人'
df.drop(['CORP_NAME'], axis=1, inplace=True)
# df.fillna(0, inplace=True)

2021-06-08 11:28:20.218 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.121s


In [80]:
df

,TRADE_NAME,ORG_CODE,SC_COMPANY,TRADE_TIMES,TRADE_VOLUMN,TYPE,RATING,BVD_RATE
0,上海上港保税仓储管理有限公司,91310115MA1K4AN53M,上海上港保税仓储管理有限公司,15.0,5.396253e+09,货主,NaN,NaN
1,上海东方嘉盛供应链管理有限公司,91310000677818840H,上海东方嘉盛供应链管理有限公司,110.0,2.889655e+08,货主,NaN,NaN
2,上海东泽供应链管理有限公司,91310000301610611H,上海东泽供应链管理有限公司,8.0,7.140588e+06,货主,NaN,NaN
3,上海东泽供应链管理有限公司,91310000301610611H,中茄实业（上海）有限公司,1.0,2.976282e+05,货主,NaN,NaN
4,上海中储临港物流有限公司,91310115795650129K,上海中储临港物流有限公司,25.0,4.299722e+09,货主,NaN,NaN
...,...,...,...,...,...,...,...,...
3127,上海飞机制造有限公司,91310000132612172J,"Toray Composite Materials America, Inc.",0.0,NaN,境外收发货人,None,NaN
3128,上海飞机制造有限公司,91310000132612172J,Transtar Metals,0.0,NaN,境外收发货人,None,NaN
3129,上海飞机制造有限公司,91310000132612172J,Triumph Thermal Systems,0.0,NaN,境外收发货人,None,NaN
3130,上海飞机制造有限公司,91310000132612172J,Tronair.Inc,0.0,NaN,境外收发货人,None,NaN


# model

In [111]:
sql = '''select * from {} where iscurrent =1 '''.format('BD_RISK_DETAIL_SUPPLYCHAIN_SC1')
df_sc = Read_Oracle().read_oracle(sql=sql, database='dbods')

2021-06-08 12:28:57.538 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.184s


In [114]:
df_sc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3132 entries, 0 to 3131
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ID                  3132 non-null   int64         
 1   TRADE_NAME          3132 non-null   object        
 2   ORG_CODE            3132 non-null   object        
 3   SC_COMPANY          3132 non-null   object        
 4   TRADE_TIMES         3132 non-null   int64         
 5   TRADE_VOLUMN        3132 non-null   float64       
 6   TYPE                3132 non-null   object        
 7   RATING              3132 non-null   object        
 8   BVD_RATE            3132 non-null   float64       
 9   STAFF_NUM           3132 non-null   float64       
 10  SHAREHOLDER_INCOME  3132 non-null   float64       
 11  INCOME              3132 non-null   float64       
 12  CHECK_TIME          3132 non-null   datetime64[ns]
 13  ISCURRENT           3132 non-null   int64       

In [116]:
init_score = 80
# 后期尝试加入k-means进行聚类，目前先人工打标签
trade_times_bin = [0,1, 3, 10, 50, 999999999999]
trade_times_score = {'[1, 3)': -3, '[3, 10)': -2, '[10, 50)': 1, '[50, 999999999999)': 5,'[0,1)':0}
trade_volumn_bin = [0,1, 100000, 1000000, 10000000, 999999999999]
trade_volumn_score = {'[1, 100000)': -3, '[100000, 1000000)': -2, '[1000000, 10000000)': 1,
                          '[10000000, 999999999999)': 5,'[0,1)':0}
staff_bin = [0,1, 100, 500, 1000, 999999999999]
staff_score = {'[1, 100)': -3, '[100, 500)': -2, '[500, 1000)': 1, '[1000, 999999999999)': 5,'[0,1)':0}
income_bin = [0,1, 100000, 1000000, 10000000, 999999999999]
income_score = {'[1, 100000)': -5, '[100000, 1000000)': -2, '[1000000, 10000000)': 1,
                          '[10000000, 999999999999)': 5,'[0,1)':0}
sincome_bin = [0, 1,100000, 1000000, 10000000, 999999999999]
sincome_score = {'[1, 100000)': -5, '[100000, 1000000)': -2, '[1000000, 10000000)': 1,
                          '[10000000, 999999999999)': 5,'[0,1)':0}
rating_score = {'AAA': 10, 'AA': 9, 'A': 8, 'BBB': 7, 'BB': 6, 'B': 5, '0': 0}
bvd_rate = {'1.0': 10, '2.0': 8, '3.0': 5, '4.0': 0, '5.0': -1, '0.0': 0}

In [117]:
df_sc['RATING'] = df_sc['RATING'].map(str)
df_sc['RATING'] = df_sc['RATING'].map(rating_score)

df_sc['BVD_RATE'] = df_sc['BVD_RATE'].map(str)
df_sc['BVD_RATE'] = df_sc['BVD_RATE'].map(bvd_rate)

df_sc['TRADE_TIMES'] = df_sc['TRADE_TIMES'].map(float)
df_sc['TRADE_TIMES'] = pd.cut(df_sc['TRADE_TIMES'], bins=trade_times_bin, right=False)
df_sc['TRADE_TIMES'] = df_sc['TRADE_TIMES'].map(str).map(trade_times_score)

df_sc['TRADE_VOLUMN'] = df_sc['TRADE_VOLUMN'].map(float)
df_sc['TRADE_VOLUMN'] = pd.cut(df_sc['TRADE_VOLUMN'], bins=trade_volumn_bin, right=False)
df_sc['TRADE_VOLUMN'] = df_sc['TRADE_VOLUMN'].map(str).map(trade_volumn_score)

df_sc['STAFF_NUM'] = df_sc['STAFF_NUM'].map(float)
df_sc['STAFF_NUM'] = pd.cut(df_sc['STAFF_NUM'], bins=staff_bin, right=False)
df_sc['STAFF_NUM'] = df_sc['STAFF_NUM'].map(str).map(staff_score)

df_sc['INCOME'] = df_sc['INCOME'].map(float)
df_sc['INCOME'] = pd.cut(df_sc['INCOME'], bins=income_bin, right=False)
df_sc['INCOME'] = df_sc['INCOME'].map(str).map(income_score)

df_sc['SHAREHOLDER_INCOME'] = df_sc['SHAREHOLDER_INCOME'].map(float)
df_sc['SHAREHOLDER_INCOME'] = pd.cut(df_sc['SHAREHOLDER_INCOME'], bins=sincome_bin, right=False)
df_sc['SHAREHOLDER_INCOME'] = df_sc['SHAREHOLDER_INCOME'].map(str).map(sincome_score)

In [140]:
df_sc = df_sc.fillna(0)

In [141]:
df_sc['SC_TOTAL_SCORE'] = df_sc.apply(
            lambda x: x['RATING'] + x['BVD_RATE'] + x['TRADE_TIMES'] + init_score + x['TRADE_TIMES'] + x['TRADE_VOLUMN'] + x['STAFF_NUM'] + x['INCOME'] + x['SHAREHOLDER_INCOME'], axis=1)

In [142]:
df_sc['SC_TOTAL_SCORE'].unique()

array([ 87.,  71.,  81.,  77.,  95.,  72.,  75.,  79.,  83.,  74.,  92.,
        86.,  80.,  89.,  94.,  78., 100.,  91.,  85.,  98.,  88.,  96.,
        97.,  90.])

In [143]:
df_sc['SC_TOTAL_SCORE'].describe()

count    3132.000000
mean       76.870051
std         4.507494
min        71.000000
25%        72.000000
50%        77.000000
75%        80.000000
max       100.000000
Name: SC_TOTAL_SCORE, dtype: float64

# 预警

In [65]:

trade_times_score = {'[0, 3)': -3, '[3, 10)': -2, '[10, 50)': 1, '[50, 999999999999)': 5}
trade_volumn_score = {'[0, 100000)': -3, '[100000, 1000000)': -2, '[1000000, 10000000)': 1,
                          '[10000000, 999999999999)': 5}
staff_score = {'[0, 100)': -3, '[100, 500)': -2, '[500, 1000)': 1, '[1000, 999999999999)': 5}
income_score = {'[0, 100000)': -5, '[100000, 1000000)': -2, '[1000000, 10000000)': 1,
                          '[10000000, 999999999999)': 5}
sincome_score = {'[0, 100000)': -5, '[100000, 1000000)': -2, '[1000000, 10000000)': 1,
                          '[10000000, 999999999999)': 5}
rating_score = {'AAA': 10, 'AA': 9, 'A': 8, 'BBB': 7, 'BB': 6, 'B': 5, '0': 0}
bvd_rate = {'1.0': 10, '2.0': 8, '3.0': 5, '4.0': 0, '5.0': -1, '0.0': 0}

In [123]:
sql = '''select * from {} where iscurrent = 1 '''.format('BD_RISK_RESULT_SUPPLYCHAIN_SC1')
df_result = Read_Oracle().read_oracle(sql=sql, database='dbods')

2021-06-08 12:49:46.942 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 2.642s


In [13]:
df_result.columns

Index(['ID', 'TRADE_NAME', 'ORG_CODE', 'SC_COMPANY', 'TRADE_TIMES',
       'TRADE_VOLUMN', 'TYPE', 'RATING', 'BVD_RATE', 'STAFF_NUM',
       'SHAREHOLDER_INCOME', 'INCOME', 'SC_TOTAL_SCORE', 'CHECK_TIME',
       'ISCURRENT', 'LASTUPDATE'],
      dtype='object')

In [151]:
json_param=[
    {'word':'发现{}起交易次数偏低事件','col':'TRADE_TIMES','score':-5},
    {'word':'发现{}起贸易金额偏低事件','col':'TRADE_VOLUMN','score':-5},
    {'word':'发现{}起企业员工人数偏低事件','col':'TRADE_VOLUMN','score':-3},
    {'word':'发现{}起经营金额偏低事件','col':'TRADE_VOLUMN','score':-5},
    {'word':'发现{}起股东收入金额偏低事件','col':'TRADE_VOLUMN','score':-5},
    {'word':'发现{}起BVD评级异常事件','col':'TRADE_VOLUMN','score':-1},
    {'word':'发现{}起BVD破产评级异常事件','col':'TRADE_VOLUMN','score':5},
]

In [152]:
def gen_alarm(i, param):
    word = param['word']
    score = param['score']
    col = param['col']
    try:
        num = df_result[(df_result[str(col)]==score) &(df_result['ORG_CODE'] == str(i)) ].groupby(['ORG_CODE'])['ID'].count().values[0]
        word = word.format(num)
        return word
    except Exception as e:
        pass

In [153]:
def get(i):
    alarm_reason=[]
    for param in json_param:
        alarm_reason.append(gen_alarm(i, param))
    return alarm_reason

In [155]:
res=[]
for i in df_result['ORG_CODE'].unique():
    for j in get(i):
        if j is not None:
            try:
                temp={}
                temp['CHECK_TIME'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                temp['ORG_CODE'] = i
                temp['MODEL_CODE'] = 'SUPPLYCHAIN'
                temp['CHILD_MODEL_CODE'] = 'SC1'
                temp['ALARM_REASON'] = j
                temp['ALARM_NUMBER'] = int(re.findall('发现(.*?)起.*', j)[0])
                res.append(temp)
            except Exception as e:
                print(e)

In [159]:
final = pd.DataFrame(res)


In [157]:
final[final['ORG_CODE']=='91310000132612172J']

,CHECK_TIME,ORG_CODE,MODEL_CODE,CHILD_MODEL_CODE,ALARM_REASON,ALARM_NUMBER
73,2021-06-08 13:31:15,91310000132612172J,SUPPLYCHAIN,SC1,发现10起交易次数偏低事件,10
74,2021-06-08 13:31:15,91310000132612172J,SUPPLYCHAIN,SC1,发现4起BVD破产评级异常事件,4


In [158]:
df_result[df_result['ORG_CODE']=='91310000132612172J']

,ID,TRADE_NAME,ORG_CODE,SC_COMPANY,TRADE_TIMES,TRADE_VOLUMN,TYPE,RATING,BVD_RATE,STAFF_NUM,SHAREHOLDER_INCOME,INCOME,SC_TOTAL_SCORE,CHECK_TIME,ISCURRENT,LASTUPDATE
165,10844,上海飞机制造有限公司,91310000132612172J,ALERIS ROLLED PRODUCTS GERMANY GMBH,-5.0,-2.0,境外收发货人,7.0,10.0,5.0,-1.0,0.0,100.0,2021-06-08 11:33:13,1,2021-06-08 12:49:37
166,10845,上海飞机制造有限公司,91310000132612172J,AMAG ROLLING GMBH,-5.0,-2.0,境外收发货人,6.0,10.0,5.0,-1.0,-2.5,100.0,2021-06-08 11:33:13,1,2021-06-08 12:49:37
167,10846,上海飞机制造有限公司,91310000132612172J,CONSTELLIUM ISSOIRE,-5.0,3.0,境外收发货人,7.0,-1.0,5.0,-1.0,-2.5,100.0,2021-06-08 11:33:13,1,2021-06-08 12:49:37
168,10847,上海飞机制造有限公司,91310000132612172J,CONSTELLIUM ROLLED PRODUCTS RAVENSWOOD LLC,-5.0,-2.0,境外收发货人,0.0,0.0,0.0,0.0,0.0,88.0,2021-06-08 11:33:13,1,2021-06-08 12:49:37
169,10848,上海飞机制造有限公司,91310000132612172J,CYTEC ENGINEERED MATERIALS INC.,-5.0,3.0,境外收发货人,7.0,0.0,-2.0,0.0,0.0,98.0,2021-06-08 11:33:13,1,2021-06-08 12:49:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3127,12523,上海飞机制造有限公司,91310000132612172J,Goodrich test,0.0,0.0,境外收发货人,0.0,0.0,0.0,0.0,0.0,100.0,2021-06-08 11:33:13,1,2021-06-08 12:49:37
3128,12524,上海飞机制造有限公司,91310000132612172J,Goodyear Singapore Tyres,0.0,0.0,境外收发货人,0.0,0.0,0.0,0.0,0.0,100.0,2021-06-08 11:33:13,1,2021-06-08 12:49:37
3129,12525,上海飞机制造有限公司,91310000132612172J,HANG YUE TONG COMPANY LIMITED,0.0,0.0,境外收发货人,0.0,0.0,0.0,0.0,0.0,100.0,2021-06-08 11:33:13,1,2021-06-08 12:49:37
3130,12526,上海飞机制造有限公司,91310000132612172J,HARDINGE ASIA LIMITED,0.0,0.0,境外收发货人,0.0,0.0,0.0,0.0,0.0,100.0,2021-06-08 11:33:13,1,2021-06-08 12:49:37


# 供应链总分

In [162]:
SC1_RESULT = Read_Oracle().read_oracle(
            sql=""" select * from {} where org_code = '{}' and iscurrent = 1 """.format('BD_RISK_RESULT_SUPPLYCHAIN_SC1','91310000132612172J'),
            database='dbods')

2021-06-08 14:15:48.367 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 2.632s


In [165]:
SC1_RESULT.groupby('ORG_CODE')['SC_TOTAL_SCORE'].mean()[0]

99.95976154992549

# 人工标签

In [39]:
from sklearn.cluster import KMeans

In [50]:
list1= df['TRADE_VOLUMN'].values.reshape(-1,1)

In [51]:
clf = KMeans(n_clusters=3)  #给定类别个数为3
s = clf.fit(list1)

In [54]:
print ('聚类中心',clf.cluster_centers_)

聚类中心 [[2.01477626e+07]
 [1.89163032e+10]
 [2.46797952e+09]]


In [56]:
print ('每个样本所属的簇',clf.labels_ ) #每个点的分类)

每个样本所属的簇 [2 0 0 ... 0 0 0]


In [57]:
print (clf.inertia_ )#每个点到其簇的质心的距离之和

5.81676059376079e+19
